# ✅ Tensor de entrada  

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día
tensors_dias = []
# Diccionario para mapear fechas a índices en tensors_dias
fechas_indices = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Procesamiento normal...
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos
        df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']] = scaler.fit_transform(df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']])
    
        # Convertir a tensor y cambiar forma
        ghi_tensor = torch.tensor(df_dia['GHI'].values).view(-1, 1)
        air_temp_tensor = torch.tensor(df_dia['air_temp'].values).view(-1, 1)
        cloud_opacity_tensor = torch.tensor(df_dia['cloud_opacity'].values).view(-1, 1)
    
        # Concatenar y cambiar forma a [1, 288, 3]
        dia_tensor = torch.cat((ghi_tensor, air_temp_tensor, cloud_opacity_tensor), dim=1).unsqueeze(0)
    
        # Agregar el tensor del día a la lista
        tensors_dias.append(dia_tensor)
        # Agregar la fecha al diccionario
        fechas_indices[fecha] = len(tensors_dias) - 1

# Verificar si hay días para concatenar
if tensors_dias:
    # Concatenar todos los tensores de días en un tensor [n, 288, 3]
    tensor_final = torch.cat(tensors_dias, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 3])
    # Almacenar el tensor en un archivo
    np.save('../RNA/data_training/input_tensor.npy', tensor_final)
    with open('../RNA/data_training/indice_fechas_input.json', 'w') as fp:
        json.dump(fechas_indices, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


torch.Size([364, 288, 3])


# ✅ Tensor SFV

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []
# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos de interés
        df_dia['Producción_fotovoltaica_SFV'] = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV']])
    
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(df_dia['Producción_fotovoltaica_SFV'].values).view(1, -1, 1)
        
        # Agregar el tensor del día a la lista
        tensors_SFV.append(produccion_tensor)
        # Registrar la fecha en el diccionario de índices
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('../RNA/data_training/tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('../RNA/data_training/indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


torch.Size([364, 288, 1])


## Crear un tensor de entrada con segundo semestre de 2023

In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
# df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')
# filtrar datos de solcast de 01/07/2023 a 31/12/2023
df_solcast_2023_2 = df_solcast[df_solcast['Date'] >= '2023-07-01']

In [6]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')
# filtrar datos de solcast de 01/07/2023 a 31/12/2023
df_solcast_2023_2 = df_solcast[df_solcast['Date'] >= '2023-07-01']

# Obtener fechas únicas
fechas_unicas = df_solcast_2023_2['Date'].unique()

# Lista para guardar los tensores de cada día
tensors_dias = []
# Diccionario para mapear fechas a índices en tensors_dias
fechas_indices = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288:
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
    
        # Normalizar los datos de interés
        variables_normalizar = ['GHI', 'air_temp', 'cloud_opacity']
        df_dia = row_day_solcast[variables_normalizar]
        df_dia = scaler.fit_transform(df_dia)
    
        # Convertir a tensor y cambiar forma
        dia_tensor = torch.tensor(df_dia).view(1, 288, -1)  # El tensor ya tiene forma [1, 288, n_variables]
    
        # Agregar el tensor del día a la lista
        tensors_dias.append(dia_tensor)
        # Agregar la fecha al diccionario
        fechas_indices[fecha] = len(tensors_dias) - 1

# Verificar si hay días para concatenar
if tensors_dias:
    # Concatenar todos los tensores de días en un tensor [n, 288, n_variables]
    tensor_final = torch.cat(tensors_dias, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, n_variables])
    # Almacenar el tensor en un archivo
    np.save('../RNA/data_training/input_tensor_2023-2.npy', tensor_final)
    with open('../RNA/data_training/indice_fechas_input_2023-2.json', 'w') as fp:
        json.dump(fechas_indices, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


torch.Size([185, 288, 3])


# Tensores de entrada con fecha  📆


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# crear columna Date* a partir de la columna Date
df_solcast ['Date*'] = df_solcast ['Date'] 

# quitar caracteres especiales y espacios
df_solcast ['Date*'] = df_solcast ['Date*'].str.replace(' ', '')
df_solcast ['Date*'] = df_solcast ['Date*'].str.replace('-', '')
print(df_solcast ['Date*'])


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
# df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
# df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día
tensors_dias = []
# Diccionario para mapear fechas a índices en tensors_dias
fechas_indices = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Procesamiento normal...
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
            'date': row_day_solcast['Date*']
        })
    
        # Normalizar los datos
        df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp', 'date']] = scaler.fit_transform(df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp', 'date']])
    
        # Convertir a tensor y cambiar forma
        ghi_tensor = torch.tensor(df_dia['GHI'].values).view(-1, 1)
        air_temp_tensor = torch.tensor(df_dia['air_temp'].values).view(-1, 1)
        cloud_opacity_tensor = torch.tensor(df_dia['cloud_opacity'].values).view(-1, 1)
        date_tensor = torch.tensor(df_dia['date'].values).view(-1, 1)
    
        # Concatenar y cambiar forma a [1, 288, 3]
        dia_tensor = torch.cat((ghi_tensor, air_temp_tensor, cloud_opacity_tensor, date_tensor), dim=1).unsqueeze(0)
    
        # Agregar el tensor del día a la lista
        tensors_dias.append(dia_tensor)
        # Agregar la fecha al diccionario
        fechas_indices[fecha] = len(tensors_dias) - 1

# Verificar si hay días para concatenar
if tensors_dias:
    # Concatenar todos los tensores de días en un tensor [n, 288, 3]
    tensor_final = torch.cat(tensors_dias, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 3])
    # Almacenar el tensor en un archivo
    np.save('input_tensor.npy', tensor_final)
    with open('indice_fechas_input.json', 'w') as fp:
        json.dump(fechas_indices, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


## Tensor SFV con fecha  📆

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# crear columna Date* a partir de la columna Date
df_SFV ['Date**'] = df_SFV ['Date'] 

# quitar caracteres especiales y espacios
df_SFV ['Date**'] = df_SFV ['Date**'].str.replace(' ', '')
df_SFV ['Date**'] = df_SFV ['Date**'].str.replace('-', '')
print(df_SFV ['Date**'])

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []
# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'date': row_day_fronius['Date**'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos de interés
        df_dia['Producción_fotovoltaica_SFV', 'date'] = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV', 'date']])
    
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(df_dia['Producción_fotovoltaica_SFV', 'date'].values).view(1, -1, 1)
        
        # Agregar el tensor del día a la lista
        tensors_SFV.append(produccion_tensor)
        # Registrar la fecha en el diccionario de índices
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json  # Importa el módulo json para la serialización

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# crear columna Date** a partir de la columna Date
df_SFV['Date**'] = df_SFV['Date'].str.replace(' ', '').replace('-', '', regex=True)

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []

# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV[df_SFV['Date'] == fecha]

    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'date': row_day_fronius['Date**'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
        
        # Normalizar los datos de interés
        normalized_values = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV', 'date']])
        df_dia[['Producción_fotovoltaica_SFV', 'date']] = normalized_values
        
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(normalized_values).view(1, -1, 2)  # Ajusta esta línea según la estructura deseada
        
        tensors_SFV.append(produccion_tensor)
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")



# ******************************************************************************

# Tensor de entrada Dividido ✂️

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import json 

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día
tensors_dias = []
# Diccionario para mapear fechas a índices en tensors_dias
fechas_indices = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Procesamiento normal...
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos
        df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']] = scaler.fit_transform(df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']])
    
        # Convertir a tensor y cambiar forma
        ghi_tensor = torch.tensor(df_dia['GHI'].values).view(-1, 1)
        air_temp_tensor = torch.tensor(df_dia['air_temp'].values).view(-1, 1)
        cloud_opacity_tensor = torch.tensor(df_dia['cloud_opacity'].values).view(-1, 1)
    
        # Concatenar y cambiar forma a [1, 288, 3]
        dia_tensor = torch.cat((ghi_tensor, air_temp_tensor, cloud_opacity_tensor), dim=1).unsqueeze(0)
    
        # Agregar el tensor del día a la lista
        tensors_dias.append(dia_tensor)
        # Agregar la fecha al diccionario
        fechas_indices[fecha] = len(tensors_dias) - 1

# Verificar si hay días para concatenar
if tensors_dias:
    # Concatenar todos los tensores de días en un tensor [n, 288, 3]
    tensor_final = torch.cat(tensors_dias, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 3])
    # Almacenar el tensor en un archivo
    np.save('all_input_tensor.npy', tensor_final)
    with open('indice_fechas_all_input.json', 'w') as fp:
        json.dump(fechas_indices, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


# Dividir los datos en conjuntos de entrenamiento y prueba 90-10

In [ ]:
import torch

# Calcular el tamaño de los datos de entrenamiento y test
num_datos = tensor_final.size(0)  # Número total de días
num_entrenamiento = int(num_datos * 0.9)  # El 90% de los datos

# Dividir los tensores
tensor_entrada_entrenamiento = tensor_final[:num_entrenamiento]  # Los primeros 90%
tensor_entrada_test = tensor_final[num_entrenamiento:]  # El resto (10%)

print(tensor_entrada_entrenamiento.shape)  # Salida esperada: torch.Size([n*0.9, 288, 3])
print(tensor_entrada_test.shape)  # Salida esperada: torch.Size([n*0.1, 288, 3])

# Almacenar el tensor en un archivo
np.save('../RNA/data_training/tensor_entrada_entrenamiento.npy', tensor_entrada_entrenamiento)
np.save('../RNA/data_training/tensor_entrada_test.npy', tensor_entrada_test)


# Agregar un diccionario con los indices para cada fecha 

#### Revisión 8 de abril 


# ****

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import json

# Cargar los datos
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

fechas_unicas = df_solcast['Date'].unique()

tensors_dias = []
fechas_indices = {}

scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV[df_SFV['Date'] == fecha]

    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)

        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })

        df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']] = scaler.fit_transform(df_dia[['GHI', 'Producción_fotovoltaica_SFV', 'clearsky_ghi', 'cloud_opacity', 'air_temp']])

        ghi_tensor = torch.tensor(df_dia['GHI'].values).view(-1, 1)
        air_temp_tensor = torch.tensor(df_dia['air_temp'].values).view(-1, 1)
        cloud_opacity_tensor = torch.tensor(df_dia['cloud_opacity'].values).view(-1, 1)

        dia_tensor = torch.cat((ghi_tensor, air_temp_tensor, cloud_opacity_tensor), dim=1).unsqueeze(0)

        tensors_dias.append(dia_tensor)
        fechas_indices[fecha] = len(tensors_dias) - 1

if tensors_dias:
    tensor_final = torch.cat(tensors_dias, dim=0)
    
    # Generar índices aleatorios y dividir
    indices = np.arange(tensor_final.size(0))
    indices_entrenamiento, indices_pruebas = train_test_split(indices, test_size=0.1, random_state=42)
    
    tensor_entrenamiento = tensor_final[indices_entrenamiento]
    tensor_pruebas = tensor_final[indices_pruebas]
    
    fechas_entrenamiento = {fecha: i for i, fecha in enumerate(fechas_indices.keys()) if i in indices_entrenamiento}
    fechas_pruebas = {fecha: i for i, fecha in enumerate(fechas_indices.keys()) if i in indices_pruebas}
    
    np.save('../RNA/data_training/tensor_entrenamiento.npy', tensor_entrenamiento)
    np.save('../RNA/data_test/tensor_pruebas.npy', tensor_pruebas)
    
    with open('../RNA/data_training/indice_fechas_entrenamiento.json', 'w') as fp:
        json.dump(fechas_entrenamiento, fp)
    
    with open('../RNA/data_test/indice_fechas_pruebas.json', 'w') as fp:
        json.dump(fechas_pruebas, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


## Tensor SFV

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []
# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    # Verificar que el día tiene 288 registros
    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        # Resetear y ordenar por hora
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        # Crear DataFrame para el día actual
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        # Normalizar los datos de interés
        df_dia['Producción_fotovoltaica_SFV'] = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV']])
    
        # Convertir la producción a tensor y cambiar forma
        produccion_tensor = torch.tensor(df_dia['Producción_fotovoltaica_SFV'].values).view(1, -1, 1)
        
        # Agregar el tensor del día a la lista
        tensors_SFV.append(produccion_tensor)
        # Registrar la fecha en el diccionario de índices
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

# Concatenar todos los tensores de días en un tensor [n, 288, 1]
if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    print(tensor_final.shape)  # Salida esperada: torch.Size([n, 288, 1])
    # Almacenar el tensor en un archivo
    np.save('tensor_SFV.npy', tensor_final)
    # Almacenar el mapeo de fechas a índices como JSON
    with open('indice_fechas_SFV.json', 'w') as fp:
        json.dump(indice_fechas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")


# ***

In [6]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import json

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener fechas únicas
fechas_unicas = df_solcast['Date'].unique()

# Lista para guardar los tensores de cada día de la producción fotovoltaica
tensors_SFV = []
# Diccionario para mapear fechas a índices en tensors_SFV
indice_fechas_SFV = {}

# Escalador MinMax
scaler = MinMaxScaler()

for fecha in fechas_unicas:
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == fecha]
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == fecha]

    if len(row_day_solcast) == 288 and len(row_day_fronius) == 288:
        row_day_solcast = row_day_solcast.sort_values(by='Time').reset_index(drop=True)
        row_day_fronius = row_day_fronius.reset_index(drop=True)
    
        df_dia = pd.DataFrame({
            'Time': row_day_solcast['Time'],
            'GHI': row_day_solcast['GHI'],
            'air_temp': row_day_solcast['air_temp'],
            'cloud_opacity': row_day_solcast['cloud_opacity'],
            'Producción_fotovoltaica_SFV': row_day_fronius['Producción_fotovoltaica_SFV'],
            'clearsky_ghi': row_day_solcast['clearsky_ghi'],
        })
    
        df_dia['Producción_fotovoltaica_SFV'] = scaler.fit_transform(df_dia[['Producción_fotovoltaica_SFV']])
    
        produccion_tensor = torch.tensor(df_dia['Producción_fotovoltaica_SFV'].values).view(1, -1, 1)
        
        tensors_SFV.append(produccion_tensor)
        indice_fechas_SFV[fecha] = len(tensors_SFV) - 1

if tensors_SFV:
    tensor_final = torch.cat(tensors_SFV, dim=0)
    
    # Generar índices aleatorios y dividir
    indices = np.arange(tensor_final.size(0))
    indices_entrenamiento, indices_pruebas = train_test_split(indices, test_size=0.1, random_state=42)
    
    tensor_entrenamiento_SFV = tensor_final[indices_entrenamiento]
    tensor_pruebas_SFV = tensor_final[indices_pruebas]
    
    # Extraer fechas correspondientes a los índices de entrenamiento y pruebas
    fechas_entrenamiento_SFV = {fecha: i for i, fecha in enumerate(indice_fechas_SFV.keys()) if indice_fechas_SFV[fecha] in indices_entrenamiento}
    fechas_pruebas_SFV = {fecha: i for i, fecha in enumerate(indice_fechas_SFV.keys()) if indice_fechas_SFV[fecha] in indices_pruebas}
    
    # Guardar los tensores y los índices de las fechas
    np.save('../RNA/data_training/tensor_entrenamiento_SFV.npy', tensor_entrenamiento_SFV)
    np.save('../RNA/data_test/tensor_pruebas_SFV.npy', tensor_pruebas_SFV)
    
    with open('../RNA/data_training/indice_fechas_entrenamiento_SFV.json', 'w') as fp:
        json.dump(fechas_entrenamiento_SFV, fp)
    
    with open('../RNA/data_test/indice_fechas_pruebas_SFV.json', 'w') as fp:
        json.dump(fechas_pruebas_SFV, fp)
else:
    print("No hay suficientes datos para crear el tensor final.")
